# COMPARISION OF TTS MODELS FOR LATENCY

## AI4BHARAT - PARLER

In [ ]:
pip install git+https://github.com/huggingface/parler-tts.git

In [ ]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
description_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)

In [ ]:
import time

In [17]:
prompt = "இந்த உலகம் மிகவும் அழகானது, இதில் நாம் ஒருவருக்கொருவர் உதவிசெய்து வாழ்ந்தால், வாழ்க்கை மேலும் அர்த்தமுள்ளதாகும்."
description = "A calm, warm female voice with medium pitch and a steady speed."


In [ ]:

description_input_ids = description_tokenizer(description, return_tensors="pt").to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").to(device)

start_time = time.time()

generation = model.generate(input_ids=description_input_ids.input_ids, attention_mask=description_input_ids.attention_mask, prompt_input_ids=prompt_input_ids.input_ids, prompt_attention_mask=prompt_input_ids.attention_mask)
audio_arr = generation.cpu().numpy().squeeze()
sf.write("indic_tts_out.wav", audio_arr, model.config.sampling_rate)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Generation time: {elapsed_time:.2f} seconds")


## SARVAM AI TTS

In [19]:
import requests
import base64
import time

API_KEY = "db88ef57-4a66-4b3b-877d-61d7586481cf"
text = prompt
url = "https://api.sarvam.ai/text-to-speech"

payload = {
    "inputs": [text],
    "target_language_code": "hi-IN",
    "speaker": "meera",
    "pitch": 0,
    "pace": 1.0,
    "loudness": 1.0,
    "speech_sample_rate": 16000,
    "enable_preprocessing": False,
    "model": "bulbul:v1"
}

headers = {
    "Content-Type": "application/json",
    "api-subscription-key": API_KEY
}

In [20]:

start_time=time.time()
response = requests.post(url, json=payload, headers=headers)
response.raise_for_status()
result = response.json()
audio_b64 = result["audios"][0]
audio_bytes = base64.b64decode(audio_b64)
with open("output_sarvam.wav", "wb") as f:
    f.write(audio_bytes)
print("Audio saved as output.wav")

end_time=time.time()
print(f"Latency of Sarvam ai is {end_time-start_time}")


Audio saved as output.wav
Latency of Sarvam ai is 1.0472426414489746


In [21]:
text

'இந்த உலகம் மிகவும் அழகானது, இதில் நாம் ஒருவருக்கொருவர் உதவிசெய்து வாழ்ந்தால், வாழ்க்கை மேலும் அர்த்தமுள்ளதாகும்.'

## SMALLEST AI

In [ ]:
text="""बिल्कुल! यहाँ 1000 अक्षरों का हिंदी गद्यांश है: भारत एक विशाल और विविधताओं से भरा देश है। यहाँ अनेक धर्म, भाषाएँ, संस्कृतियाँ और परंपराएँ एक साथ निवास करती हैं। भारत की सभ्यता विश्व की सबसे प्राचीन सभ्यताओं में से एक मानी जाती है। यहाँ के लोग अपने अतिथि सत्कार, सहिष्णुता और भाईचारे के लिए प्रसिद्ध हैं। भारत में अनेक पर्व-त्योहार मनाए जाते हैं, जिनमें दीपावली, होली, ईद, क्रिसमस, बैसाखी, पोंगल आदि प्रमुख हैं। यहाँ की प्राकृतिक सुंदरता भी अद्भुत है-हिमालय की बर्फीली चोटियाँ, गंगा-यमुना जैसी पवित्र नदियाँ, राजस्थान के रेगिस्तान, केरल के बैकवाटर और सुंदरवन के जंगल, सब मिलकर भारत को अनोखा बनाते हैं। भारतीय इतिहास में कई महान व्यक्तित्व हुए हैं, जैसे महात्मा गांधी, जवाहरलाल नेहरू, सरदार पटेल, भगत सिंह, रानी लक्ष्मीबाई आदि। स्वतंत्रता संग्राम में अनेक वीरों ने अपने प्राणों की आहुति दी। आज़ाद भारत विज्ञान, तकनीक, खेल, शिक्षा और कला के क्षेत्र में निरंतर प्रगति कर रहा है। यहाँ की फिल्में, संगीत, नृत्य और साहित्य ने विश्वभर में अपनी पहचान बनाई है। भारतीय लोकतंत्र विश्व का सबसे बड़ा लोकतंत्र है, जहाँ हर नागरिक को अपने मताधिकार का प्रयोग करने की स्वतंत्रता है। यहाँ की विविधता में एकता ही इसकी सबसे बड़ी ताकत है, जो भारत को अन्य देशों से अलग बनाती है।"""

In [ ]:
text[:500]

In [ ]:
import requests

SMALLEST_API_KEY="sk_b9dd40a425cbce74d8e2d397b2802823"

url = "https://waves-api.smallest.ai/api/v1/lightning/get_speech"

payload = {
    "voice_id": "emily",
    "text": prompt,
    "sample_rate": 24000,
    "add_wav_header": True
}

headers = {
    "Authorization": f"Bearer {SMALLEST_API_KEY}",
    "Content-Type": "application/json"
}


In [ ]:
response = requests.request("POST", url, json=payload, headers=headers)

start_time = time.time()
if response.status_code == 200:
    with open("output_smallest.wav", "wb") as audio_file:
        audio_file.write(response.content)
else:
  print(response.status_code)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Generation time: {elapsed_time:.2f} seconds")

# TEXT TRANSLATION LATENCY

## google translator

In [ ]:
pip install googletrans==4.0.0-rc1

In [ ]:
from googletrans import Translator, LANGUAGES
import time
translator = Translator()

text = """बिल्कुल! यहाँ एक लगभग 1000 शब्दों का हिंदी गद्यांश है, जिसमें भारतीय संस्कृति, भाषा, शिक्षा, विज्ञान, समाज, और पर्यावरण जैसे विषयों को समाहित किया गया है। यह टेक्स्ट आपके किसी भी NLP, TTS, या अनुवाद मॉडल के लिए आदर्श है।

भारत एक विविधताओं से भरा देश है, जहाँ विभिन्न भाषाएँ, धर्म, संस्कृतियाँ और परंपराएँ एक साथ मिलकर एक अद्वितीय समाज का निर्माण करती हैं। भारतीय संस्कृति की जड़ें हजारों वर्षों पुरानी हैं और यह आज भी अपनी प्राचीनता और आधुनिकता का सुंदर संगम प्रस्तुत करती है। यहाँ की भाषाएँ, साहित्य, कला, संगीत, नृत्य, वास्तुकला, और विज्ञान ने न केवल भारत को बल्कि पूरे विश्व को समृद्ध किया है।

भारतीय भाषाओं की बात करें तो हिंदी, जो कि भारत की राजभाषा है, करोड़ों लोगों द्वारा बोली और समझी जाती है। हिंदी साहित्य का इतिहास भी अत्यंत समृद्ध है। कबीर, तुलसीदास, सूरदास, प्रेमचंद, महादेवी वर्मा, हरिवंश राय बच्चन जैसे कवि और लेखक हिंदी साहित्य को नई ऊँचाइयों तक ले गए। हिंदी भाषा ने न केवल साहित्य में, बल्कि सिनेमा, रेडियो, टेलीविजन और इंटरनेट के माध्यम से भी जनमानस में अपनी गहरी छाप छोड़ी है।

भारत में शिक्षा का भी एक गौरवशाली इतिहास रहा है। प्राचीन काल में तक्षशिला, नालंदा, विक्रमशिला जैसे विश्वविद्यालयों में देश-विदेश से विद्यार्थी अध्ययन के लिए आते थे। आज भी भारत में शिक्षा को अत्यधिक महत्व दिया जाता है। यहाँ की शिक्षा प्रणाली में समय-समय पर सुधार होते रहे हैं, जिससे विद्यार्थियों को आधुनिक विज्ञान, तकनीक, गणित, चिकित्सा, और अन्य विषयों में उत्कृष्ट ज्ञान प्राप्त हो सके। डिजिटल शिक्षा और ऑनलाइन लर्निंग ने शिक्षा के क्षेत्र में क्रांति ला दी है, जिससे दूरदराज के क्षेत्रों में भी ज्ञान की पहुँच संभव हुई है।

भारतीय समाज में परिवार, रिश्ते, और सामाजिक मूल्यों का विशेष स्थान है। यहाँ संयुक्त परिवार की परंपरा रही है, जिसमें एक ही छत के नीचे कई पीढ़ियाँ साथ रहती हैं। हालाँकि, समय के साथ समाज में बदलाव आया है और अब एकल परिवारों की संख्या बढ़ रही है, लेकिन फिर भी भारतीय समाज में आपसी सहयोग, सम्मान, और सहिष्णुता की भावना बनी हुई है। त्योहारों का भी यहाँ विशेष महत्व है। दीपावली, होली, ईद, क्रिसमस, गुरुपर्व, पोंगल, ओणम आदि त्योहार पूरे देश में हर्षोल्लास के साथ मनाए जाते हैं, जो सामाजिक एकता और सांस्कृतिक विविधता का प्रतीक हैं।

भारतीय विज्ञान और तकनीक ने भी विश्व में अपनी पहचान बनाई है। आर्यभट्ट, भास्कराचार्य, चरक, सुश्रुत जैसे वैज्ञानिकों और चिकित्सकों ने प्राचीन भारत में गणित, ज्योतिष, चिकित्सा, और सर्जरी के क्षेत्र में अद्वितीय योगदान दिया। आधुनिक भारत में भी इसरो, डीआरडीओ, आईआईटी, आईआईएससी जैसी संस्थाएँ विज्ञान और तकनीक के क्षेत्र में उत्कृष्ट कार्य कर रही हैं। भारत का चंद्रयान और मंगलयान मिशन, डिजिटल इंडिया, स्टार्टअप इंडिया जैसी पहलें देश को तकनीकी दृष्टि से सशक्त बना रही हैं।

पर्यावरण संरक्षण भी आज के समय की एक महत्वपूर्ण आवश्यकता है। भारत में वनों, नदियों, पर्वतों, और जैव विविधता का अनमोल खजाना है। लेकिन बढ़ती आबादी, शहरीकरण, और औद्योगीकरण के कारण पर्यावरण पर दबाव बढ़ा है। जलवायु परिवर्तन, वायु और जल प्रदूषण, वन्य जीवों की घटती संख्या जैसी समस्याएँ सामने आ रही हैं। ऐसे में, हमें पर्यावरण संरक्षण के लिए जागरूक होना होगा। वृक्षारोपण, जल संरक्षण, प्लास्टिक का कम उपयोग, और स्वच्छता जैसे कार्यों में सभी को भागीदारी निभानी चाहिए।

भारतीय युवाओं में अपार ऊर्जा, उत्साह और प्रतिभा है। वे शिक्षा, खेल, विज्ञान, कला, संगीत, साहित्य, और उद्यमिता के क्षेत्र में देश का नाम रोशन कर रहे हैं। आज के युवा डिजिटल युग के साथ कदम से कदम मिलाकर चल रहे हैं। सोशल मीडिया, इंटरनेट, और मोबाइल तकनीक ने उनकी सोच और दृष्टिकोण को वैश्विक बना दिया है। वे न केवल अपने देश की समस्याओं को समझते हैं, बल्कि उनके समाधान के लिए भी तत्पर रहते हैं।

भारतीय लोकतंत्र भी विश्व में अपनी एक अलग पहचान रखता है। यहाँ की संसद, न्यायपालिका, और कार्यपालिका संविधान के अनुसार कार्य करती हैं। नागरिकों को अभिव्यक्ति की स्वतंत्रता, शिक्षा का अधिकार, समानता, और न्याय जैसे मौलिक अधिकार प्राप्त हैं। चुनावों के माध्यम से जनता अपनी सरकार चुनती है, जिससे लोकतंत्र की जड़ें और मजबूत होती हैं।

अंत में, यह कहना उचित होगा कि भारत की विविधता, संस्कृति, भाषा, विज्ञान, और समाज ने उसे विश्व में एक विशिष्ट स्थान दिलाया है। यहाँ के लोग अपने मूल्यों, परंपराओं, और आधुनिक सोच के साथ आगे बढ़ रहे हैं। हमें अपने देश की इस अनमोल विरासत को संजोकर रखना चाहिए और आने वाली पीढ़ियों को भी इसके महत्व से अवगत कराना चाहिए। इसी में भारत की सच्ची प्रगति और समृद्धि निहित है।

अगर आपको किसी विशेष विषय या शैली में हिंदी टेक्स्ट चाहिए तो कृपया बताएं!"""

start_time = time.time()

translated = translator.translate(text, dest='en')

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Generation time: {elapsed_time:.2f} seconds")

In [ ]:
print(f"Translated: {translated.text}")

## SARVAM AI TRANSlATION

In [ ]:
text[0:200]

In [ ]:
import requests

API_KEY = "db88ef57-4a66-4b3b-877d-61d7586481cf"
url = "https://api.sarvam.ai/translate"

def split_text(text, max_len=1000):
    return [text[i:i+max_len] for i in range(0, len(text), max_len)]

def translate_chunk(chunk):
    payload = {
        "input": chunk,
        "source_language_code": "auto",
        "target_language_code": "en-IN",
        "speaker_gender": "Female",
        "mode": "formal",
        "model": "mayura:v1",
        "enable_preprocessing": False,
    }
    headers = {
        "Content-Type": "application/json",
        "api-subscription-key": API_KEY
    }
    
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    result = response.json()
    return result["translated_text"]


start_time=time.time()
chunks = split_text(text, max_len=500)
translated_chunks = [translate_chunk(chunk) for chunk in chunks]
full_translation = " ".join(translated_chunks)
end_time=time.time()
print(f"Latency of Sarvam Ai for translation is {end_time-start_time}")
print(full_translation)

# SPEECH TO TEXT FOR LOW LATENCY

## SARVAM AI

In [4]:
import requests
import io

In [5]:
SARVAM_AI_API="db88ef57-4a66-4b3b-877d-61d7586481cf"
api_url = "https://api.sarvam.ai/speech-to-text-translate"

headers = {"api-subscription-key": SARVAM_AI_API}

data = {
    "model": "saaras:v2",
    "with_diarization": False,
}

In [6]:

from pydub import AudioSegment


def split_audio(audio_path, chunk_duration_ms):
    """
    Splits an audio file into smaller chunks of specified duration.

    Args:
        audio_path (str): Path to the audio file to be split.
        chunk_duration_ms (int): Duration of each chunk in milliseconds.

    Returns:
        list: A list of AudioSegment objects representing the audio chunks.
    """
    audio = AudioSegment.from_file(audio_path)
    chunks = []
    if len(audio) > chunk_duration_ms:

        for i in range(0, len(audio), chunk_duration_ms):
            chunks.append(audio[i : i + chunk_duration_ms])
    else:

        chunks.append(audio)
    return chunks

In [7]:
def translate_audio(
    audio_file_path, api_url, headers, data, chunk_duration_ms=30 * 1000
):
    """
    Translates audio into text with optional diarization and timestamps.

    Args:
        audio_file_path (str): Path to the audio file.
        api_url (str): API endpoint URL for Speech-to-Text and Translate.
        headers (dict): Headers for API authentication.
        data (dict): Payload containing model and other options like diarization.
        chunk_duration_ms (int): Duration of each audio chunk in milliseconds.

    Returns:
        dict: Collated response containing the transcript and word-level timestamps.
    """
    chunks = split_audio(audio_file_path, chunk_duration_ms)
    responses = []


    for idx, chunk in enumerate(chunks):
        # Export the chunk to a BytesIO object (in-memory binary stream)
        chunk_buffer = io.BytesIO()
        chunk.export(chunk_buffer, format="wav")
        chunk_buffer.seek(0)  # Reset the pointer to the start of the stream

        # Prepare the file for the API request
        files = {"file": ("audiofile.wav", chunk_buffer, "audio/wav")}

        try:
            # Make the POST request to the API
            response = requests.post(api_url, headers=headers, files=files, data=data)
            if response.status_code == 200 or response.status_code == 201:
                print(f"Chunk {idx} POST Request Successful!")
                response_data = response.json()
                transcript = response_data.get("transcript", "")
                responses.append({"transcript": transcript})
            else:
                # Handle failed requests
                print(
                    f"Chunk {idx} POST Request failed with status code: {response.status_code}"
                )
                print("Response:", response.text)
        except Exception as e:
            # Handle any exceptions during the request
            print(f"Error processing chunk {idx}: {e}")
        finally:
            # Ensure the buffer is closed after processing
            chunk_buffer.close()

    # Collate the transcriptions from all chunks
    collated_transcript = " ".join([resp["transcript"] for resp in responses])
    collated_response = {
        "transcript": collated_transcript,
        "language": response_data.get("language_code"),
    }
    return collated_response


In [ ]:
audio_file_path="/content/New-Recording.wav"

start_time=time.time()

translation = translate_audio(audio_file_path, api_url, headers, data)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Generation time: {elapsed_time:.2f} seconds")

translation

## WISHPHER

In [ ]:
!pip install openai-whisper
!apt-get install ffmpeg

In [ ]:
import whisper

model = whisper.load_model("medium")


start_time=time.time()

result = model.transcribe("/content/New-Recording.wav", task="translate")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Generation time: {elapsed_time:.2f} seconds")


print("Detected language:", result["language"])
print("English transcript:", result["text"])
